In [ ]:
import onnx
import torch
import numpy as np
import pandas as pd  
import copy
import pprint
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

from torch import nn

import sys
sys.path.append('/Users/khoanguyen-cp/gmu/network_properties')

from models.test_models import ProphecyPaperNetwork, TestModel
from models.acasxu_1_1 import Acasxu1_1
from models.utils import attach_relu_activation_hook, attach_layer_output_hook, get_layers_info
from models.utils import turn_bool_activation_to_int, turn_bool_activation_to_str

from algorithms.decision_procedure import MarabouCoreDP

## 1. Prepare model and data

In [ ]:
model = Acasxu1_1()
model.load_state_dict(torch.load('../models/acasxu_1_1.pt'))

In [ ]:
acas_train = np.empty([384221,5],dtype=float)
acas_train_labels = np.zeros(384221,dtype=int)

def read_inputs_from_file(inputFile):
  global acas_train, acas_train_labels, num
  with open(inputFile) as f:
    lines = f.readlines()
    print(len(lines), "examples")
    acas_train = np.empty([len(lines),5],dtype=float)
    acas_train_labels = np.zeros(len(lines),dtype=int)

    for l in range(len(lines)):
      # This is to remove the useless 1 at the start of each string. Not sure why that's there.
      k = [float(stringIn) for stringIn in lines[l].split(',')] 
      
      # acas_train[l+num] = np.zeros(5,dtype=float) 
      # we're asuming that everything is 2D for now. The 1 is just to keep numpy happy.
      if len(k) > 5:
        lab = int(k[5])
        #if ((lab == 0) or (lab == 2)):
        #  lab = 0
        #else:
        #  lab = 1
        acas_train_labels[l+num] = lab

      count = 0
      for i in range(0,5):
        #print(count)
        acas_train[l+num][i] = k[i]
        #print(k[i])

In [ ]:
num = 0
read_inputs_from_file('../datasets/clusterinACAS_0_shrt.csv')
print(acas_train.shape)
print(acas_train_labels.shape)
print(acas_train[:5])
print(acas_train_labels[:5])

In [ ]:
def create_df(inputs, predicted_labels, true_labels, activation_signature):
  data = []
  for index, input_data in enumerate(inputs):
    data_point = { 
      "input": input_data, 
      "true_label": true_labels[index], 
      "predicted_label": predicted_labels[index].item(),
    }
    data_point_full_signature = {} 
    for name, layer_activation in activation_signature.items():
      data_point[name] = json.dumps(layer_activation[index])
      data_point_full_signature[name] = layer_activation[index]
      
    data_point['full_signature_str'] = json.dumps(data_point_full_signature)
    data.append(data_point)
  return pd.DataFrame(data)

_act_handles, activation_signature = attach_relu_activation_hook(model)  
outputs = model(torch.tensor(acas_train, dtype=torch.float32))
predicted_labels = torch.argmin(outputs, dim=1)
activation_signature = turn_bool_activation_to_int(activation_signature, to_list=True)
df = create_df(acas_train, predicted_labels, acas_train_labels, activation_signature)
df.head(10)

## 2. Sampling

In [ ]:
def initialize_activation_counts(model, sample):
  _act_handles, activation_signature = attach_relu_activation_hook(model)  
  X = torch.tensor(sample, dtype=torch.float)
  _logits = model(X)
  
  activation_signature = turn_bool_activation_to_int(activation_signature)
  for layer_name, activations in activation_signature.items():
    activation_signature[layer_name] = np.zeros_like(activations[0])
    
  return activation_signature.copy()

In [ ]:
y_class = 0

# get all data points with predicted class y
class_df = df[df['predicted_label'] == y_class]

ranges = []
for i in range(5):
  min_val = class_df['input'].apply(lambda x: x[i]).min()
  max_val = class_df['input'].apply(lambda x: x[i]).max()
  ranges.append([min_val, max_val])

# initialize the activation counts dictionary using the activation signature dictionary
# cus they should have the same structure 
activation_counts = initialize_activation_counts(model, [[0.62, 0.1, 0.2, 0.47, -0.48]])  

# calculate activation probability of each neuron in the network
for index, row in class_df.iterrows():
  full_signature = json.loads(row['full_signature_str'])
  for layer, activation in full_signature.items():
    # update activation count
    activation_counts[layer] += np.array(activation)
    
activation_probabilities = {}
for layer, neuron_act_counts in activation_counts.items():
  activation_probabilities[layer] = neuron_act_counts/len(class_df)
    
np.set_printoptions(formatter={'float': '{: 0.8f}'.format})
activation_probabilities

In [ ]:
candidate = activation_probabilities.copy()
for layer, activation_probs in candidate.items():
  new_activation = [
    "ON" if prob == 1 else ("OFF" if prob == 0 else "--")
    for prob in activation_probs
  ]
  candidate[layer] = new_activation
print(candidate)

In [ ]:
specification_for_classes = {
  0: [(np.array([[-1, 1, 0, 0, 0]]), np.array([0])),
      (np.array([[-1, 0, 1, 0, 0]]), np.array([0])),
      (np.array([[-1, 0, 0, 1, 0]]), np.array([0])),
      (np.array([[-1, 0, 0, 0, 1]]), np.array([0]))],
  
  1: [(np.array([[1, -1, 0, 0, 0]]), np.array([0])),
      (np.array([[0, -1, 1, 0, 0]]), np.array([0])),
      (np.array([[0, -1, 0, 1, 0]]), np.array([0])),
      (np.array([[0, -1, 0, 0, 1]]), np.array([0]))],
  
  2: [(np.array([[1, 0, -1, 0, 0]]), np.array([0])),
      (np.array([[0, 1, -1, 0, 0]]), np.array([0])),
      (np.array([[0, 0, -1, 1, 0]]), np.array([0])),
      (np.array([[0, 0, -1, 0, 1]]), np.array([0]))],
  
  3: [(np.array([[1, 0, 0, -1, 0]]), np.array([0])),
      (np.array([[0, 1, 0, -1, 0]]), np.array([0])),
      (np.array([[0, 0, 1, -1, 0]]), np.array([0])),
      (np.array([[0, 0, 0, -1, 1]]), np.array([0]))],
  
  4: [(np.array([[1, 0, 0, 0, -1]]), np.array([0])),
      (np.array([[0, 1, 0, 0, -1]]), np.array([0])),
      (np.array([[0, 0, 1, 0, -1]]), np.array([0])),
      (np.array([[0, 0, 0, 1, -1]]), np.array([0]))],
}

In [ ]:
dp = MarabouCoreDP()
dp.solve(candidate, model, ranges, specification_for_classes[y_class])